In [1]:
import sys

sys.path.append("..")
from piece import piece

piece.start(should_send_to_score=True)

In [2]:
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments

from soundmining_tools.supercollider_client import SupercolliderClient
from soundmining_tools.modular.synth_player import SynthNote
from soundmining_tools.generative import *
from enum import Enum
import math
from soundmining_tools.sequencer import Sequencer, SequenceNote
from soundmining_tools.modular.instrument import NodeId
from soundmining_tools.ui.ui_piece import UiPieceBuilder
from ipycanvas import Canvas, hold_canvas
from common import *


PART_LENGTH = 89

setup_piece()

class PotHitScratchPart:
    LOW_ROOM_EFFECT = 0
    LOW_BACKGROUND_EFFECT = 2
    LOW_CLEAN = 4
    MIDDLE_ROOM_EFFECT = 6
    MIDDLE_BACKGROUND_EFFECT = 8
    MIDDLE_CLEAN = 10
    HIGH_ROOM_EFFECT = 12
    HIGH_BACKGROUND_EFFECT = 14
    HIGH_CLEAN = 16

    @classmethod
    def make_low_effect(cls, start_time: float, duration: float) -> list[SynthNote]:

        room_effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(room_effect)
                .stereo_volume(piece.control_instruments.static_control(0.8))
                .stereo_convolution_reverb(LOW_POT_HIT_SCRATCH_IR, piece.control_instruments.static_control(1.0))
                .play(start_time, duration, output_bus=cls.LOW_ROOM_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(room_effect)
                .stereo_volume(piece.control_instruments.static_control(1.4))
                .play(start_time, duration, output_bus=cls.LOW_CLEAN)
        )

        background_effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())
        
        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(background_effect)
                .stereo_volume(piece.control_instruments.static_control(0.2))
                .stereo_g_verb(piece.control_instruments.static_control(1), roomsize=50, revtime=13, damping=0.8)
                .play(start_time, duration, output_bus=cls.LOW_BACKGROUND_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(background_effect)
                .stereo_volume(piece.control_instruments.static_control(1.4))
                .play(start_time, duration, output_bus=cls.LOW_CLEAN)
        )
        return [room_effect, background_effect]
    
    @classmethod
    def make_middle_effect(cls, start_time: float, duration: float) -> list[SynthNote]:
        room_effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(room_effect)
                .stereo_volume(piece.control_instruments.static_control(0.9))
                .stereo_convolution_reverb(MIDDLE_POT_HIT_SCRATCH_IR, piece.control_instruments.static_control(1.0))
                .play(start_time, duration, output_bus=cls.MIDDLE_ROOM_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(room_effect)
                .stereo_volume(piece.control_instruments.static_control(1.3))
                .play(start_time, duration, output_bus=cls.MIDDLE_CLEAN)
        )

        background_effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())
        
        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(background_effect)
                .stereo_volume(piece.control_instruments.static_control(0.2))
                .stereo_g_verb(piece.control_instruments.static_control(1), roomsize=50, revtime=13, damping=0.8)
                .play(start_time, duration, output_bus=cls.MIDDLE_BACKGROUND_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(background_effect)
                .stereo_volume(piece.control_instruments.static_control(1.3))
                .play(start_time, duration, output_bus=cls.MIDDLE_CLEAN)
        )

        return [room_effect, background_effect]

    @classmethod
    def make_high_effect(cls, start_time: float, duration: float) -> list[SynthNote]:
        room_effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(room_effect)
                .stereo_volume(piece.control_instruments.static_control(0.7))
                .stereo_convolution_reverb(HIGH_POT_HIT_SCRATCH_IR, piece.control_instruments.static_control(1.0))
                .play(start_time, duration, output_bus=cls.HIGH_ROOM_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(room_effect)
                .stereo_volume(piece.control_instruments.static_control(1.2))
                .play(start_time, duration, output_bus=cls.HIGH_CLEAN)
        )

        background_effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(background_effect)
                .stereo_volume(piece.control_instruments.static_control(0.2))
                .stereo_g_verb(piece.control_instruments.static_control(1), roomsize=50, revtime=13, damping=0.8)
                .play(start_time, duration, output_bus=cls.HIGH_BACKGROUND_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(background_effect)
                .stereo_volume(piece.control_instruments.static_control(1.2))
                .play(start_time, duration, output_bus=cls.HIGH_CLEAN)
        )

        return [room_effect, background_effect]
    
    @classmethod
    def play_low_part(cls, start_time: float) -> list[SequenceNote]:                
        end_time = start_time + PART_LENGTH
        current_time = start_time + random_range(1, 3)
        effects = cls.make_low_effect(start_time, PART_LENGTH)
        notes = []
        while current_time < (end_time - 5):
            number_of_notes, group_notes = PotHitScratchGroup.play_low_group(current_time, effects)
            notes.extend(group_notes)
            match number_of_notes:                
                case 1:
                    current_time += (8 * random_range(0.85, 1.15))
                case 2:
                    current_time += (13 * random_range(0.85, 1.15))
        return notes

    @classmethod
    def play_middle_part(cls, start_time: float) -> list[SequenceNote]:
        end_time = start_time + PART_LENGTH
        current_time = start_time + random_range(1, 3)
        effects = cls.make_middle_effect(start_time, PART_LENGTH)
        notes = []
        while current_time < (end_time - 5):
            number_of_notes, group_notes = PotHitScratchGroup.play_middle_group(current_time, effects)
            notes.extend(group_notes)
            match number_of_notes:
                case 1:
                    current_time += (5 * random_range(0.85, 1.15))
                case 2:
                    current_time += (8 * random_range(0.85, 1.15))
                case 3:
                    current_time += (13 * random_range(0.85, 1.15))
        return notes
    
    @classmethod
    def play_high_part(cls, start_time: float) -> list[SequenceNote]:
        end_time = start_time + PART_LENGTH
        current_time = start_time + random_range(1, 3)
        effects = cls.make_high_effect(start_time, PART_LENGTH)
        notes = []
        while current_time < (end_time - 5):
            number_of_notes, group_notes = PotHitScratchGroup.play_high_group(current_time, effects)
            notes.extend(group_notes)
            match number_of_notes:
                case 1:
                    current_time += (5 * random_range(0.85, 1.15))
                case 2:
                    current_time += (8 * random_range(0.85, 1.15))
                case 3:
                    current_time += (13 * random_range(0.85, 1.15))
        return notes


class ShortPotHitRattlePart:
    LOW_ROOM_EFFECT = 18
    LOW_BACKGROUND_EFFECT = 20
    LOW_CLEAN = 22
    MIDDLE_ROOM_EFFECT = 24
    MIDDLE_BACKGROUND_EFFECT = 26
    MIDDLE_CLEAN = 28
    HIGH_ROOM_EFFECT = 30
    HIGH_BACKGROUND_EFFECT = 32
    HIGH_CLEAN = 34

    @classmethod
    def make_low_effect(cls, start_time: float, duration: float) -> list[SynthNote]:            
        room_effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(room_effect)
                .stereo_volume(piece.control_instruments.static_control(0.3))                
                .stereo_convolution_reverb(LOW_SHORT_POT_HIT_RATTLE_IR, piece.control_instruments.static_control(1.0))                
                .play(start_time, duration, output_bus=cls.LOW_ROOM_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(room_effect)
                .stereo_volume(piece.control_instruments.static_control(1.7))
                .play(start_time, duration, output_bus=cls.LOW_CLEAN)
        )

        background_effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())
        
        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(background_effect)
                .stereo_volume(piece.control_instruments.static_control(0.2))
                .stereo_g_verb(piece.control_instruments.static_control(1), roomsize=50, revtime=13, damping=0.8)
                .play(start_time, duration, output_bus=cls.LOW_BACKGROUND_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(background_effect)
                .stereo_volume(piece.control_instruments.static_control(1.7))
                .play(start_time, duration, output_bus=cls.LOW_CLEAN)
        )
        return [room_effect, background_effect]
    
    @classmethod
    def make_middle_effect(cls, start_time: float, duration: float) -> list[SynthNote]:            
        room_effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(room_effect)
                .stereo_volume(piece.control_instruments.static_control(0.4))
                .stereo_convolution_reverb(MIDDLE_SHORT_POT_HIT_RATTLE_IR, piece.control_instruments.static_control(1.0))
                .play(start_time, duration, output_bus=cls.MIDDLE_ROOM_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(room_effect)
                .stereo_volume(piece.control_instruments.static_control(0.6))
                .play(start_time, duration, output_bus=cls.MIDDLE_CLEAN)
        )

        background_effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())
        
        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(background_effect)
                .stereo_volume(piece.control_instruments.static_control(0.2))
                .stereo_g_verb(piece.control_instruments.static_control(1), roomsize=50, revtime=13, damping=0.8)
                .play(start_time, duration, output_bus=cls.MIDDLE_BACKGROUND_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(room_effect)
                .stereo_volume(piece.control_instruments.static_control(0.6))
                .play(start_time, duration, output_bus=cls.MIDDLE_CLEAN)
        )

        return [room_effect, background_effect]
    
    @classmethod
    def make_high_effect(cls, start_time: float, duration: float) -> list[SynthNote]:            
        room_effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(room_effect)
                .stereo_volume(piece.control_instruments.static_control(0.3))
                .stereo_convolution_reverb(HIGH_SHORT_POT_HIT_RATTLE_IR, piece.control_instruments.static_control(1.0))
                .play(start_time, duration, output_bus=cls.HIGH_ROOM_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(room_effect)
                .stereo_volume(piece.control_instruments.static_control(0.4))
                .play(start_time, duration, output_bus=cls.HIGH_CLEAN)
        )

        background_effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())
        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(background_effect)
                .stereo_volume(piece.control_instruments.static_control(0.1))
                .stereo_g_verb(piece.control_instruments.static_control(1), roomsize=50, revtime=13, damping=0.8)
                .play(start_time, duration, output_bus=cls.HIGH_BACKGROUND_EFFECT)
        )

        (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(background_effect)
                .stereo_volume(piece.control_instruments.static_control(0.4))
                .play(start_time, duration, output_bus=cls.HIGH_CLEAN)
        )
        return [room_effect, background_effect]
    
    @classmethod
    def play_low_part(cls, start_time: float) -> list[SequenceNote]:
        end_time = start_time + PART_LENGTH
        current_time = start_time + random_range(1, 3)
        effects = cls.make_low_effect(start_time, PART_LENGTH)
        notes = []
        while current_time < (end_time - 5):
            number_of_notes, group_notes = ShortPotHitRattleGroup.play_low_group(current_time, effects)
            notes.extend(group_notes)
            match number_of_notes:
                case 1:
                    current_time += (5 * random_range(0.85, 1.15))
                case 2:
                    current_time += (8 * random_range(0.85, 1.15))
        return notes

    @classmethod
    def play_middle_part(cls, start_time: float) -> list[SequenceNote]:
        end_time = start_time + PART_LENGTH
        current_time = start_time + random_range(1, 3)
        effects = cls.make_middle_effect(start_time, PART_LENGTH)
        notes = []
        while current_time < (end_time - 5):
            number_of_notes, group_notes = ShortPotHitRattleGroup.play_middle_group(current_time, effects)
            notes.extend(group_notes)
            match number_of_notes:
                case 1:
                    current_time += (5 * random_range(0.85, 1.15))
                case 2:
                    current_time += (8 * random_range(0.85, 1.15))
                case 3:
                    current_time += (13 * random_range(0.85, 1.15))
        return notes

    @classmethod
    def play_high_part(cls, start_time: float) -> list[SequenceNote]:
        end_time = start_time + PART_LENGTH
        current_time = start_time + random_range(1, 3)
        effects = cls.make_high_effect(start_time, PART_LENGTH)
        notes = []
        while current_time < (end_time - 5):
            number_of_notes, group_notes = ShortPotHitRattleGroup.play_high_group(current_time, effects)
            notes.extend(group_notes)
            match number_of_notes:                
                case 1:
                    current_time += (5 * random_range(0.85, 1.15))                
                case 2:
                    current_time += (8 * random_range(0.85, 1.15))                
                case 3:
                    current_time += (13 * random_range(0.85, 1.15))
        return notes
        

class PartHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def handle_note(self, patch_arguments: PatchArguments) -> None:  
        
        match patch_arguments.midi_note:
            case 48:
                ShortPotHitRattlePart.play_low_part(patch_arguments.start)
            case 49:
                ShortPotHitRattlePart.play_middle_part(patch_arguments.start)
            case 50:
                ShortPotHitRattlePart.play_high_part(patch_arguments.start)
            case 51:
                PotHitScratchPart.play_low_part(patch_arguments.start)                
            case 52:
                PotHitScratchPart.play_middle_part(patch_arguments.start)
            case 53:
                PotHitScratchPart.play_high_part(patch_arguments.start)

class GroupHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def make_effect(self, start_time: float, ir: str, duration: float = 5) -> SynthNote:
        effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        reverb_effect = (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.4))
                .stereo_convolution_reverb(ir, piece.control_instruments.static_control(1.0))
                .play(start_time, duration)
        )

        clean_effect = (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.6))
                .play(start_time, duration)
        )
        return effect
    
    def handle_note(self, patch_arguments: PatchArguments) -> None:
        start_delay = 0.5
        match patch_arguments.midi_note:
            case 48:
                effect = self.make_effect(patch_arguments.start, "ir6")
                ShortPotHitRattleGroup.play_low_group(patch_arguments.start + start_delay, effect)
            case 49:
                effect = self.make_effect(patch_arguments.start, "ir1")
                ShortPotHitRattleGroup.play_middle_group(patch_arguments.start + start_delay, effect)
            case 50:
                effect = self.make_effect(patch_arguments.start, "ir7")
                ShortPotHitRattleGroup.play_high_group(patch_arguments.start + start_delay, effect)
            case 51:
                effect = self.make_effect(patch_arguments.start, "ir8")
                PotHitScratchGroup.play_low_group(patch_arguments.start + start_delay, effect)
            case 52:
                effect = self.make_effect(patch_arguments.start, "ir2")
                PotHitScratchGroup.play_middle_group(patch_arguments.start + start_delay, effect)
            case 53:
                effect = self.make_effect(patch_arguments.start, "ir4")
                PotHitScratchGroup.play_high_group(patch_arguments.start + start_delay, effect)

my_handler = PartHandler(piece.supercollider_client)
piece.receiver.set_note_handler(my_handler)

class PartType(Enum):
    LOW_SHORT_POT_HIT_RATTLE = 1
    MIDDLE_SHORT_POT_HIT_RATTLE = 2
    HIGH_SHORT_POT_HIT_RATTLE = 3
    LOW_POT_HIT_SCRATCH = 4
    MIDDLE_POT_HIT_SCRATCH = 5
    HIGH_POT_HIT_SCRATCH = 6

part_type_chain = MarkovChain({
   (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2
   },
   (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2
   },
   (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0
   },
   (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2
   },
   (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2
   },
   (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0
   },
   (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0
   },
   (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0
   },
   (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0
   }
}, (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH))

from typing import Callable

def step_handler(i: int, start: float) -> list[SequenceNote]:
    notes = []
    for part_type in part_type_chain.next():
        start_time = start + random_range(0.3, 0.7)
        match part_type:
            case PartType.LOW_SHORT_POT_HIT_RATTLE:
                notes.extend(ShortPotHitRattlePart.play_low_part(start_time))
            case PartType.MIDDLE_SHORT_POT_HIT_RATTLE:
                notes.extend(ShortPotHitRattlePart.play_middle_part(start_time))
            case PartType.HIGH_SHORT_POT_HIT_RATTLE:
                notes.extend(ShortPotHitRattlePart.play_high_part(start_time))
            case PartType.LOW_POT_HIT_SCRATCH:
                notes.extend(PotHitScratchPart.play_low_part(start_time))
            case PartType.MIDDLE_POT_HIT_SCRATCH:
                notes.extend(PotHitScratchPart.play_middle_part(start_time))
            case PartType.HIGH_POT_HIT_SCRATCH:
                notes.extend(PotHitScratchPart.play_high_part(start_time))
    return notes

NUMBER_OF_SEQUENCE_GROUPS = 3
#NUMBER_OF_SEQUENCES_IN_GROUP = 2
TOTAL_NUMBER_OF_SEQUENCES = 8
PAUSE_LENGTH = 13

SEQUENCE_LENGTHS = [
    PART_LENGTH, PART_LENGTH - 13, PART_LENGTH - 13,
    PART_LENGTH, PART_LENGTH
]
last_sequenser = (
    Sequencer(len(SEQUENCE_LENGTHS))
        .add_step_handler(step_handler)
        .next_time_handler(lambda i: SEQUENCE_LENGTHS[i] * random_range(0.85, 1.0))
)


notes = last_sequenser.generate(0)
#notes = []

if piece.synth_player.should_send_to_score:
    piece.synth_player.supercollider_score.make_score_file("concrete-music-11-v2.txt")

ui_piece = UiPieceBuilder().add_notes(notes).build()

piece_duration = ui_piece.get_duration()

piece_stats = {"total": piece_duration, "total minutes": piece_duration / 60.0, "tracks": len(ui_piece.tracks)}

min_freq = 0
max_freq = 0

for track in ui_piece.tracks:
    track_duration = 0
    for note in track.notes:
        track_duration = max(track_duration, note.start + note.duration)
        min_freq = min(min_freq, note.freq)
        max_freq = max(max_freq, note.freq)

    piece_stats[track.track_name] = track_duration

display(piece_stats)

TRACK_HEIGHT = 100
NOTE_SCALE_FACTOR = 3
HEIGHT_INDENT = 80

ui_width = 200 + (piece_duration * NOTE_SCALE_FACTOR)
ui_height = TRACK_HEIGHT * len(ui_piece.tracks)

from ipywidgets import Output

canvas = Canvas(width=ui_width, height=ui_height)

out = Output()


@out.capture()
def handle_mouse_down(x, y):
    canvas.flush()
    print("Mouse down event:", x, y)


canvas.on_mouse_down(handle_mouse_down)
canvas.global_alpha = 0.7

display(canvas)


with hold_canvas():

    canvas.clear()
    for track_index, track in enumerate(sorted(ui_piece.tracks, key=lambda tr: tr.track_name)):
        canvas.font = "14px sans-serif"
        canvas.fill_style = "Black"
        canvas.fill_text(
            track.track_name, x=20, y=(track_index * TRACK_HEIGHT) + HEIGHT_INDENT
        )
        canvas.stroke_style = "Black"
        canvas.stroke_lines(
            [
                (150, (track_index * TRACK_HEIGHT) + 10),
                (150, ((track_index * TRACK_HEIGHT) + TRACK_HEIGHT - 10)),
            ]
        )
        for note in track.notes:
            relative_note = (note.freq - min_freq) / (max_freq - min_freq)
            startx = 200 + (note.start * NOTE_SCALE_FACTOR)
            starty = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
            )
            peakx = 200 + (note.start + (note.duration * note.peak)) * NOTE_SCALE_FACTOR
            peaky = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
                - 5
            )
            endx = 200 + (note.start + note.duration) * NOTE_SCALE_FACTOR
            endy = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
            )
            canvas.stroke_style = note.color
            canvas.stroke_lines([(startx, starty), (peakx, peaky), (endx, endy)])

import ipywidgets as widgets

stop_button = widgets.Button(description="Stop")
status = widgets.Output()
display(stop_button, status)
with status:
    print("Playing")

def stop_playback(b):
    piece.reset()
    canvas.clear()
    status.clear_output()
    with status:
        print("Playback stopped")


stop_button.on_click(stop_playback)



{'total': 374.4142369870133,
 'total minutes': 6.240237283116888,
 'tracks': 15,
 'Short rattle variant 2 high': 238.20455202401627,
 'Short rattle variant 1 high': 238.07367102702568,
 'Pot Hit Short High': 229.364532237827,
 'Long Scratch high': 372.7803584030732,
 'Pot hit long high': 372.59166011239677,
 'Short rattle variant 2 low': 374.4142369870133,
 'Pot Hit Short Low': 374.24963173614424,
 'Short rattle variant 1 low': 360.81857881574877,
 'Long Scratch low': 284.68514696252015,
 'Pot hit long low': 299.0918500864201,
 'Long Scratch middle': 228.62584795849799,
 'Pot hit long middle': 228.39802459479665,
 'Short rattle variant 2 middle': 274.5411565088683,
 'Short rattle variant 1 middle': 298.4323229329653,
 'Pot Hit Short Middle': 298.1163570198278}

Canvas(height=1500, width=1323)

Button(description='Stop', style=ButtonStyle())

Output()

In [3]:
piece.stop()